In [7]:
import json
def read_json(file_path): 
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [8]:
results_base = read_json('../outputs/results_base.json')
results_oversampling = read_json('../outputs/results_oversampling.json')
results_focal_loss = read_json('../outputs/results_focal_loss.json')
results_class_weights = read_json('../outputs/results_class_weights.json')
results_cost_sensitive = read_json('../outputs/results_cost_sensitive.json')

In [9]:
import pandas as pd
import numpy as np

def extract_metrics(results_dict, positive_label="1"):
    cm = results_dict["confusion_matrix"]
    tn, fp = cm[0]
    fn, tp = cm[1]

    # Recall for each class
    recall_pos = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    recall_neg = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    gmean = np.sqrt(recall_pos * recall_neg)

    return {
        "Accuracy": results_dict["accuracy"],
        "Balanced Accuracy": results_dict["balanced_accuracy"],
        "Precision (Crisis)": results_dict["classification_report"][positive_label]["precision"],
        "Recall (Crisis)": recall_pos,
        "F1-score (Crisis)": results_dict["classification_report"][positive_label]["f1-score"],
        "G-Mean": gmean
    }


In [10]:
results_table = pd.DataFrame.from_dict(
    {
        "Baseline LSTM": extract_metrics(results_base),
        "Window Oversampling": extract_metrics(results_oversampling),
        "Focal Loss": extract_metrics(results_focal_loss),
        "Class Weights": extract_metrics(results_class_weights),
        "Adaptive Cost-Sensitive": extract_metrics(results_cost_sensitive),
    },
    orient="index"
)

# Optional: nicer formatting
results_table = results_table.round(3)

results_table


,Accuracy,Balanced Accuracy,Precision (Crisis),Recall (Crisis),F1-score (Crisis),G-Mean
Baseline LSTM,0.200,0.565,0.091,1.0,0.167,0.361
Window Oversampling,0.640,0.804,0.182,1.0,0.308,0.780
Focal Loss,0.520,0.739,0.143,1.0,0.250,0.692
Class Weights,0.762,0.875,0.167,1.0,0.286,0.866
Adaptive Cost-Sensitive,0.714,0.850,0.143,1.0,0.250,0.837


In [11]:
print(
    results_table.to_latex(
        float_format="%.3f",
        caption="Performance comparison of imbalance-handling techniques",
        label="tab:imbalance_results"
    )
)

\begin{table}
\caption{Performance comparison of imbalance-handling techniques}
\label{tab:imbalance_results}
\begin{tabular}{lrrrrrr}
\toprule
 & Accuracy & Balanced Accuracy & Precision (Crisis) & Recall (Crisis) & F1-score (Crisis) & G-Mean \\
\midrule
Baseline LSTM & 0.200 & 0.565 & 0.091 & 1.000 & 0.167 & 0.361 \\
Window Oversampling & 0.640 & 0.804 & 0.182 & 1.000 & 0.308 & 0.780 \\
Focal Loss & 0.520 & 0.739 & 0.143 & 1.000 & 0.250 & 0.692 \\
Class Weights & 0.762 & 0.875 & 0.167 & 1.000 & 0.286 & 0.866 \\
Adaptive Cost-Sensitive & 0.714 & 0.850 & 0.143 & 1.000 & 0.250 & 0.837 \\
\bottomrule
\end{tabular}
\end{table}

